# Advent of Code 2021: Day 4
link to puzzle [here](https://adventofcode.com/2021/day/4)

Imports

In [6]:
from dataclasses import dataclass, field
from typing import List

import numpy as np
import numpy.typing as npt

!ln -sf ../utils.py .
import utils

Load puzzle input data

In [7]:
DATA_DIR = '../data'
DAY = 4
data = utils.get_puzzle_input(day=DAY, input_dir=DATA_DIR)

Classes

In [8]:
@dataclass
class BingoBoard:
    """Class for bingo board"""
    
    # only input is a 2D board passed as a numpy array
    board: np.ndarray
    
    def __post_init__(self):
        
        # dimensions of bingo board
        self.dim = self.board.shape
        self.nrows, ncols = self.dim
    
        # initialize a marker board to track drawn numbers on the board
        self.init_marker()        
    
    def init_marker(self):
        """build an array with the same dimension as `board` with all values are `False`
        """
        self.marker = np.zeros(self.dim, dtype=bool)
    
    def update_marker(self, draws: List[int]):
        """update the marker board elements False -> True for any number in `draws` also present in `board`
        """
        self.marker = np.isin(self.board, draws)
        
    def reset_marker(self):
        """reset marker by re-initializing to all `False` values
        """
        self.init_marker()
        
    def is_winner(self):
        """determine if board is a winner by checking if the marker board contains all True
        values in any row or column 
        """
        return self.marker.all(axis=0).any() | self.marker.all(axis=1).any()
    
    def sum_of_unmarked(self):
        """return the sum of values of `board` which are `False` in `marker`
        """
        return self.board[~self.marker].sum()

Functions

In [9]:
def get_bingo_draws(data):
    """get bingo number draws from puzzle input data
    """
    return [int(draw) for draw in data[0].split(',')]

def get_bingo_board_arrays(data):
    """get bingo boards from puzzle input data
    """
    boards = []
    board = []
    for ln in data[1:]:
        if len(ln) > 0:
            board.append([int(num) for num in ln.split()])
        else:
            if len(board) > 0:
                boards.append(board)
                board = []
    return np.array(boards)

def game_has_winner(bingo_boards, draws):
    for bb in bingo_boards:
        bb.update_marker(draws)
        if bb.is_winner():
            return True
    return False

def get_min_draws_to_win(bingo_boards, draws):
    """perform binary search on a list bingo draws to find the minimum number of consecutive draws
    which will produce at least 1 winning board. Note: we assume there is at least 1 winner if all 
    numbers in `draws` are used
    """
    total_draws = len(draws)
    prev_winner_num_draws = total_draws
    curr_winner_num_draws = total_draws 
    prev_num_draws = total_draws // 2
    curr_num_draws = total_draws // 2
    
    while True:

        # check if game has a winner for current draw selection
        winner_seen = game_has_winner(bingo_boards, draws[:curr_num_draws])
        if winner_seen:
            # update the number of draws to find a winner since it has decreased  
            prev_winner_num_draws = curr_winner_num_draws
            curr_winner_num_draws = curr_num_draws        

            # reduce search range to see if a winner is found with even less draws
            prev_num_draws = curr_num_draws
            curr_num_draws = curr_winner_num_draws // 2

            # min draws must be at least current winning draws value
            winning_draws_upper_bound = curr_winner_num_draws
        else:
            
            # if n draws doesn't produce a winner, but our last winner was n + 1, 
            # then n + 1 must be the minimum number of draws to find a winner, so exit loop
            if (curr_num_draws == curr_winner_num_draws - 1):
                break

            # if no winners for current number of draws, then extend search range
            prev_num_draws = curr_num_draws
            curr_num_draws = (prev_num_draws + curr_winner_num_draws) // 2
    return curr_winner_num_draws

#### Part 1
Solution

In [11]:
draws = get_bingo_draws(data)
arrs = get_bingo_board_arrays(data)

# initialize bingo board objects
bingo_boards = [BingoBoard(arr) for arr in arrs]

# get minimum number of draws to find a winner
min_draws = get_min_draws_to_win(bingo_boards, draws)

# get board which wins after min number of draws
# NOTE: we assume there is at most 1 winner for each draw
winner = [bb for bb in bingo_boards if game_has_winner([bb], draws[:min_draws])][0]
winner.update_marker(draws[:min_draws])
print(draws[min_draws - 1] * winner.sum_of_unmarked())

29440


#### Part 2
Solution

In [64]:
# re-initialize bingo board objects so that they ar reset from part 1
bingo_boards = [BingoBoard(arr) for arr in arrs]

# get minimum number of draws to win for each board
min_draws_by_board = {i: get_min_draws_to_win([bb], draws) for i, bb in enumerate(bingo_boards)}

# find the index of the board which requires the most draws until it wins
most_draws_bb_index = max(min_draws_by_board, key=min_draws_by_board.get)

# get the bingo board which takes the most draws to win 
last_bb_to_win = bingo_boards[most_draws_bb_index]
last_bb_to_win.reset_marker()
last_bb_to_win.update_marker(draws[:winning_draw_index])

# get the winning draw
winning_draw = draws[min_draws_by_board[most_draws_bb_index] - 1]
print(last_bb_to_win.sum_of_unmarked() * winning_draw)

13884
